<a href="https://colab.research.google.com/github/AidaArredondo/Trabajo-Final/blob/main/AlmaMaterParqueadero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datetime
import csv
import os
from datetime import datetime as dt, timedelta

# Variables globales para almacenar datos
usuarios_registrados = []
vehiculos_ingresados = []
vehiculos_retirados = []
usuarios_admin = [
    {"usuario": "administrador1", "password": "123456"},
    {"usuario": "administrador2", "password": "1234567"}
]

# Configuración de zona horaria para Colombia (UTC-5)
ZONA_HORARIA_COLOMBIA = timedelta(hours=-5)

def obtener_hora_colombia():
    """Obtiene la hora actual en zona horaria de Colombia (UTC-5)"""
    hora_utc = dt.utcnow()
    hora_colombia = hora_utc + ZONA_HORARIA_COLOMBIA
    return hora_colombia

def limpiar_pantalla():
    """Limpia la pantalla de la consola"""
    os.system('cls' if os.name == 'nt' else 'clear')

def mostrar_menu_principal():
    """Muestra el menú principal del sistema"""
    print("=" * 50)
    print("    PARQUEADERO ALMA MATER")
    print("    Sistema de Gestión")
    print("=" * 50)
    print("1. Registrar Usuario")
    print("2. Ingresar Vehículo")
    print("3. Retirar Vehículo")
    print("4. Administrador")
    print("5. Salir")
    print("=" * 50)

def validar_nombre(nombre, tipo_campo):
    """Valida que el nombre o apellido cumplan con los requisitos"""
    errores = []

    if len(nombre) < 3:
        errores.append(f"El {tipo_campo} debe tener al menos 3 caracteres")

    if any(char.isdigit() for char in nombre):
        errores.append(f"El {tipo_campo} no puede contener números")

    return errores

def validar_documento(documento):
    """Valida que el documento cumpla con los requisitos"""
    errores = []

    if not documento.isdigit():
        errores.append("El documento solo puede contener números")
    elif len(documento) < 3 or len(documento) > 15:
        errores.append("El documento debe tener entre 3 y 15 dígitos")

    return errores

def validar_placa(placa):
    """Valida que la placa cumpla con los requisitos"""
    errores = []

    if len(placa) != 6:
        errores.append("La placa debe tener exactamente 6 caracteres")
    elif not (placa[:3].isalpha() and placa[3:].isdigit()):
        errores.append("La placa debe tener 3 letras seguidas de 3 números")

    return errores

def buscar_usuario_por_documento(documento):
    """Busca un usuario por su documento"""
    for usuario in usuarios_registrados:
        if usuario["documento"] == documento:
            return usuario
    return None

def buscar_vehiculo_por_placa(placa):
    """Busca un vehículo ingresado por su placa"""
    for vehiculo in vehiculos_ingresados:
        if vehiculo["placa"] == placa:
            return vehiculo
    return None

def registrar_usuario():
    """Registra un nuevo usuario en el sistema"""
    print("\n--- REGISTRAR USUARIO ---")

    while True:
        nombre = input("Ingrese el nombre: ").strip()
        errores_nombre = validar_nombre(nombre, "nombre")
        if not errores_nombre:
            break
        for error in errores_nombre:
            print(f"Error: {error}")

    while True:
        apellido = input("Ingrese el apellido: ").strip()
        errores_apellido = validar_nombre(apellido, "apellido")
        if not errores_apellido:
            break
        for error in errores_apellido:
            print(f"Error: {error}")

    while True:
        documento = input("Ingrese el documento: ").strip()
        errores_documento = validar_documento(documento)
        if not errores_documento:
            # Verificar si el documento ya está registrado
            if buscar_usuario_por_documento(documento):
                print("Error: Este documento ya está registrado")
                continue
            break
        for error in errores_documento:
            print(f"Error: {error}")

    while True:
        placa = input("Ingrese la placa del vehículo: ").strip().upper()
        errores_placa = validar_placa(placa)
        if not errores_placa:
            # Verificar si la placa ya está registrada
            placa_existente = False
            for usuario in usuarios_registrados:
                if usuario["placa"] == placa:
                    placa_existente = True
                    break
            if placa_existente:
                print("Error: Esta placa ya está registrada")
                continue
            break
        for error in errores_placa:
            print(f"Error: {error}")

    # Crear el usuario
    usuario = {
        "nombre": nombre,
        "apellido": apellido,
        "documento": documento,
        "placa": placa
    }

    usuarios_registrados.append(usuario)
    print(f"\nUsuario {nombre} {apellido} registrado exitosamente!")
    input("Presione Enter para continuar...")

def ingresar_vehiculo():
    """Registra el ingreso de un vehículo al parqueadero"""
    print("\n--- INGRESAR VEHÍCULO ---")

    documento = input("Ingrese el documento del usuario: ").strip()
    usuario = buscar_usuario_por_documento(documento)

    if not usuario:
        print("Error: Usuario no registrado")
        input("Presione Enter para continuar...")
        return

    # Verificar si el vehículo ya está en el parqueadero
    if buscar_vehiculo_por_placa(usuario["placa"]):
        print("Error: Este vehículo ya se encuentra en el parqueadero")
        input("Presione Enter para continuar...")
        return

    # Verificar disponibilidad de espacios
    if len(vehiculos_ingresados) >= 64:
        print("Error: No hay espacios disponibles en el parqueadero")
        input("Presione Enter para continuar...")
        return

    # Registrar ingreso
    hora_ingreso = obtener_hora_colombia()
    vehiculo_ingreso = {
        "documento": documento,
        "nombre": usuario["nombre"],
        "apellido": usuario["apellido"],
        "placa": usuario["placa"],
        "hora_ingreso": hora_ingreso
    }

    vehiculos_ingresados.append(vehiculo_ingreso)

    # Mostrar recibo
    print("\n" + "=" * 40)
    print("         RECIBO DE INGRESO")
    print("      PARQUEADERO ALMA MATER")
    print("=" * 40)
    print(f"Nombre: {usuario['nombre']} {usuario['apellido']}")
    print(f"Documento: {usuario['documento']}")
    print(f"Placa: {usuario['placa']}")
    print(f"Hora de ingreso: {hora_ingreso.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Espacios disponibles: {64 - len(vehiculos_ingresados)}")
    print("=" * 40)
    print("Conserve este recibo para el retiro")
    print("=" * 40)

    input("Presione Enter para continuar...")

def calcular_costo(tiempo_minutos):
    """Calcula el costo del parqueo basado en el tiempo"""
    horas_completas = tiempo_minutos // 60
    minutos_restantes = tiempo_minutos % 60

    # Calcular cuartos de hora
    cuartos_hora = 0
    if minutos_restantes > 0:
        cuartos_hora = (minutos_restantes + 14) // 15  # Redondear hacia arriba

    costo_horas = horas_completas * 7000
    costo_cuartos = cuartos_hora * 1500
    total = costo_horas + costo_cuartos

    # Pago mínimo de una hora
    if total < 7000:
        total = 7000
        costo_horas = 7000
        costo_cuartos = 0
        horas_completas = 1
        cuartos_hora = 0

    return {
        "horas_completas": horas_completas,
        "cuartos_hora": cuartos_hora,
        "costo_horas": costo_horas,
        "costo_cuartos": costo_cuartos,
        "total": total
    }

def retirar_vehiculo():
    """Registra el retiro de un vehículo y calcula el costo"""
    print("\n--- RETIRAR VEHÍCULO ---")

    placa = input("Ingrese la placa del vehículo: ").strip().upper()
    vehiculo = buscar_vehiculo_por_placa(placa)

    if not vehiculo:
        print("Error: Vehículo no encontrado en el parqueadero")
        input("Presione Enter para continuar...")
        return

    # Calcular tiempo de estancia
    hora_salida = obtener_hora_colombia()
    tiempo_estancia = hora_salida - vehiculo["hora_ingreso"]
    tiempo_minutos = int(tiempo_estancia.total_seconds() // 60)

    # Calcular costo
    costo_info = calcular_costo(tiempo_minutos)

    # Crear registro de retiro
    vehiculo_retirado = vehiculo.copy()
    vehiculo_retirado["hora_salida"] = hora_salida
    vehiculo_retirado["tiempo_minutos"] = tiempo_minutos
    vehiculo_retirado["costo_total"] = costo_info["total"]

    vehiculos_retirados.append(vehiculo_retirado)
    vehiculos_ingresados.remove(vehiculo)

    # Mostrar factura
    print("\n" + "=" * 45)
    print("            FACTURA DE PAGO")
    print("        PARQUEADERO ALMA MATER")
    print("=" * 45)
    print(f"Nombre: {vehiculo['nombre']} {vehiculo['apellido']}")
    print(f"Documento: {vehiculo['documento']}")
    print(f"Placa: {vehiculo['placa']}")
    print(f"Hora ingreso: {vehiculo['hora_ingreso'].strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Hora salida: {hora_salida.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Tiempo total: {tiempo_minutos} minutos")
    print("-" * 45)
    print("DETALLE DE COBRO:")
    print(f"Horas completas: {costo_info['horas_completas']} x $7,000 = ${costo_info['costo_horas']:,}")
    print(f"Cuartos de hora: {costo_info['cuartos_hora']} x $1,500 = ${costo_info['costo_cuartos']:,}")
    print("-" * 45)
    print(f"TOTAL A PAGAR: ${costo_info['total']:,}")
    print("=" * 45)

    input("Presione Enter para continuar...")

def validar_admin():
    """Valida las credenciales de administrador"""
    print("\n--- ACCESO ADMINISTRADOR ---")
    usuario = input("Usuario: ").strip()
    password = input("Contraseña: ").strip()

    for admin in usuarios_admin:
        if admin["usuario"] == usuario and admin["password"] == password:
            return True

    print("Credenciales incorrectas")
    input("Presione Enter para continuar...")
    return False

def mostrar_menu_admin():
    """Muestra el menú de administrador"""
    print("\n" + "=" * 50)
    print("        MÓDULO ADMINISTRADOR")
    print("=" * 50)
    print("1. Total de vehículos registrados")
    print("2. Total de vehículos retirados")
    print("3. Total de vehículos sin retirar")
    print("4. Total pago de vehículos retirados")
    print("5. Tiempo promedio de estancia")
    print("6. Lista de usuarios")
    print("7. Vehículo con tiempo máximo y mínimo")
    print("8. Exportar datos a CSV")
    print("9. Volver al menú principal")
    print("=" * 50)

def reporte_vehiculos_registrados():
    """Muestra el total de vehículos registrados"""
    total = len(usuarios_registrados)
    print(f"\nTotal de vehículos registrados: {total}")
    input("Presione Enter para continuar...")

def reporte_vehiculos_retirados():
    """Muestra el total de vehículos retirados"""
    total = len(vehiculos_retirados)
    print(f"\nTotal de vehículos retirados: {total}")
    input("Presione Enter para continuar...")

def reporte_vehiculos_sin_retirar():
    """Muestra el total de vehículos sin retirar"""
    total = len(vehiculos_ingresados)
    print(f"\nTotal de vehículos sin retirar: {total}")
    if total > 0:
        print("\nVehículos actualmente en el parqueadero:")
        for vehiculo in vehiculos_ingresados:
            tiempo_actual = obtener_hora_colombia() - vehiculo["hora_ingreso"]
            minutos = int(tiempo_actual.total_seconds() // 60)
            print(f"- {vehiculo['placa']} | {vehiculo['nombre']} {vehiculo['apellido']} | {minutos} min")
    input("Presione Enter para continuar...")

def reporte_total_pagos():
    """Muestra el total de pagos recibidos"""
    total = sum(vehiculo["costo_total"] for vehiculo in vehiculos_retirados)
    print(f"\nTotal de pagos recibidos: ${total:,}")
    print(f"Número de transacciones: {len(vehiculos_retirados)}")
    if len(vehiculos_retirados) > 0:
        promedio = total / len(vehiculos_retirados)
        print(f"Pago promedio por vehículo: ${promedio:,.0f}")
    input("Presione Enter para continuar...")

def reporte_tiempo_promedio():
    """Calcula y muestra el tiempo promedio de estancia"""
    if len(vehiculos_retirados) == 0:
        print("\nNo hay datos de vehículos retirados para calcular el promedio")
        input("Presione Enter para continuar...")
        return

    total_minutos = sum(vehiculo["tiempo_minutos"] for vehiculo in vehiculos_retirados)
    promedio_minutos = total_minutos / len(vehiculos_retirados)

    horas = int(promedio_minutos // 60)
    minutos = int(promedio_minutos % 60)

    print(f"\nTiempo promedio de estancia: {horas} horas y {minutos} minutos")
    print(f"({promedio_minutos:.1f} minutos en total)")
    input("Presione Enter para continuar...")

def reporte_lista_usuarios():
    """Muestra la lista de todos los usuarios registrados"""
    if len(usuarios_registrados) == 0:
        print("\nNo hay usuarios registrados")
        input("Presione Enter para continuar...")
        return

    print(f"\nLISTA DE USUARIOS REGISTRADOS ({len(usuarios_registrados)} usuarios):")
    print("-" * 60)
    for i, usuario in enumerate(usuarios_registrados, 1):
        print(f"{i}. {usuario['nombre']} {usuario['apellido']}")
        print(f"   Documento: {usuario['documento']} | Placa: {usuario['placa']}")
        print("-" * 60)

    input("Presione Enter para continuar...")

def reporte_tiempo_maximo_minimo():
    """Muestra los vehículos con tiempo máximo y mínimo de parqueo"""
    if len(vehiculos_retirados) == 0:
        print("\nNo hay datos de vehículos retirados")
        input("Presione Enter para continuar...")
        return

    vehiculo_max = max(vehiculos_retirados, key=lambda x: x["tiempo_minutos"])
    vehiculo_min = min(vehiculos_retirados, key=lambda x: x["tiempo_minutos"])

    print("\nVEHÍCULO CON MAYOR TIEMPO DE PARQUEO:")
    print(f"Placa: {vehiculo_max['placa']}")
    print(f"Propietario: {vehiculo_max['nombre']} {vehiculo_max['apellido']}")
    print(f"Tiempo: {vehiculo_max['tiempo_minutos']} minutos")

    print("\nVEHÍCULO CON MENOR TIEMPO DE PARQUEO:")
    print(f"Placa: {vehiculo_min['placa']}")
    print(f"Propietario: {vehiculo_min['nombre']} {vehiculo_min['apellido']}")
    print(f"Tiempo: {vehiculo_min['tiempo_minutos']} minutos")

    input("Presione Enter para continuar...")

def exportar_csv():
    """Exporta los datos a archivos CSV"""
    try:
        # Exportar usuarios registrados
        with open('usuarios_registrados.csv', 'w', newline='', encoding='utf-8') as archivo:
            campos = ['nombre', 'apellido', 'documento', 'placa']
            escritor = csv.DictWriter(archivo, fieldnames=campos)
            escritor.writeheader()
            escritor.writerows(usuarios_registrados)

        # Exportar vehículos retirados
        with open('vehiculos_retirados.csv', 'w', newline='', encoding='utf-8') as archivo:
            campos = ['nombre', 'apellido', 'documento', 'placa', 'hora_ingreso', 'hora_salida', 'tiempo_minutos', 'costo_total']
            escritor = csv.DictWriter(archivo, fieldnames=campos)
            escritor.writeheader()
            for vehiculo in vehiculos_retirados:
                vehiculo_csv = vehiculo.copy()
                vehiculo_csv['hora_ingreso'] = vehiculo['hora_ingreso'].strftime('%Y-%m-%d %H:%M:%S')
                vehiculo_csv['hora_salida'] = vehiculo['hora_salida'].strftime('%Y-%m-%d %H:%M:%S')
                escritor.writerow(vehiculo_csv)

        # Exportar vehículos actuales
        with open('vehiculos_actuales.csv', 'w', newline='', encoding='utf-8') as archivo:
            campos = ['nombre', 'apellido', 'documento', 'placa', 'hora_ingreso']
            escritor = csv.DictWriter(archivo, fieldnames=campos)
            escritor.writeheader()
            for vehiculo in vehiculos_ingresados:
                vehiculo_csv = vehiculo.copy()
                vehiculo_csv['hora_ingreso'] = vehiculo['hora_ingreso'].strftime('%Y-%m-%d %H:%M:%S')
                escritor.writerow(vehiculo_csv)

        print("\nArchivos CSV exportados exitosamente:")
        print("- usuarios_registrados.csv")
        print("- vehiculos_retirados.csv")
        print("- vehiculos_actuales.csv")

    except Exception as e:
        print(f"\nError al exportar archivos CSV: {str(e)}")

    input("Presione Enter para continuar...")

def modulo_administrador():
    """Módulo principal de administración"""
    if not validar_admin():
        return

    while True:
        limpiar_pantalla()
        mostrar_menu_admin()

        try:
            opcion = int(input("Seleccione una opción: "))

            if opcion == 1:
                reporte_vehiculos_registrados()
            elif opcion == 2:
                reporte_vehiculos_retirados()
            elif opcion == 3:
                reporte_vehiculos_sin_retirar()
            elif opcion == 4:
                reporte_total_pagos()
            elif opcion == 5:
                reporte_tiempo_promedio()
            elif opcion == 6:
                reporte_lista_usuarios()
            elif opcion == 7:
                reporte_tiempo_maximo_minimo()
            elif opcion == 8:
                exportar_csv()
            elif opcion == 9:
                break
            else:
                print("Opción no válida")
                input("Presione Enter para continuar...")

        except ValueError:
            print("Por favor, ingrese un número válido")
            input("Presione Enter para continuar...")

def main():
    """Función principal del programa"""
    while True:
        limpiar_pantalla()
        mostrar_menu_principal()

        try:
            opcion = int(input("Seleccione una opción: "))

            if opcion == 1:
                registrar_usuario()
            elif opcion == 2:
                ingresar_vehiculo()
            elif opcion == 3:
                retirar_vehiculo()
            elif opcion == 4:
                modulo_administrador()
            elif opcion == 5:
                print("\nGracias por usar el Sistema de Gestión del Parqueadero Alma Mater")
                print("¡Hasta pronto!")
                break
            else:
                print("Opción no válida. Por favor, seleccione una opción del 1 al 5.")
                input("Presione Enter para continuar...")

        except ValueError:
            print("Por favor, ingrese un número válido")
            input("Presione Enter para continuar...")

if __name__ == "__main__":
    main()

    PARQUEADERO ALMA MATER
    Sistema de Gestión
1. Registrar Usuario
2. Ingresar Vehículo
3. Retirar Vehículo
4. Administrador
5. Salir
Seleccione una opción: 1

--- REGISTRAR USUARIO ---
Ingrese el nombre: Aida Andrea 
Ingrese el apellido: Arredondo
Ingrese el documento: 1011590860
Ingrese la placa del vehículo: nit567

Usuario Aida Andrea Arredondo registrado exitosamente!
Presione Enter para continuar...
    PARQUEADERO ALMA MATER
    Sistema de Gestión
1. Registrar Usuario
2. Ingresar Vehículo
3. Retirar Vehículo
4. Administrador
5. Salir
Seleccione una opción: 2

--- INGRESAR VEHÍCULO ---
Ingrese el documento del usuario: 1011590860

         RECIBO DE INGRESO
      PARQUEADERO ALMA MATER
Nombre: Aida Andrea Arredondo
Documento: 1011590860
Placa: NIT567
Hora de ingreso: 2025-06-20 22:51:02
Espacios disponibles: 63
Conserve este recibo para el retiro
Presione Enter para continuar...3
    PARQUEADERO ALMA MATER
    Sistema de Gestión
1. Registrar Usuario
2. Ingresar Vehículo
3. Re